In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
sys.path.append("../../")
from plot_OpSims import get_metric_medians
from script_utils import get_opsim_areas

In [3]:
# import maf python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [4]:
# import convenience functions
import sys 
sys.path.append("../../../LSST_OpSim")
from Scripts_NBs.opsimUtils import *

## Load the results
We first load the results of the run into memory. We will read jointly the FBS v1.5 and FBS v1.6 results, and will differentiate at the time of plotting.

In [5]:
NSIDE = 64
your_username = "rjassef"
folder_mafoutput = "EM5_{0:d}_v2".format(NSIDE)
resultDbPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)
metricDataPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

In [6]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [7]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSID

In [8]:
#Rearrange the bundleDicts_raw dictionary so that the keys are always the same as for the first run.
dbRuns = list(bundleDicts_raw.keys())
Keys = list(bundleDicts_raw[dbRuns[0]].keys())
bundleDicts = dict()
for runName in dbRuns:
    bundleDicts[runName] = dict()
    Keys_raw = bundleDicts_raw[runName]
    for Key in Keys:
        for Key_raw in Keys_raw:
            if Key[1]==Key_raw[1]:
                bundleDicts[runName][Key] = bundleDicts_raw[runName][Key_raw]

In [9]:
# check keys
dbRuns = list(resultDbs.keys())
bd_keys = list(bundleDicts[dbRuns[0]].keys())
print(bd_keys)

[(1, 'EM5_r'), (2, 'EM5_g'), (3, 'EM5_u'), (4, 'EM5_i'), (5, 'EM5_y'), (6, 'EM5_z')]


## Process the metric results for each OpSim and save the results in a table.

Note that in general, the rank will be from worst (0%) to best (100%)

In [10]:
sys.path.append("../../Filter_Curves/")
from lam_eff import lam_eff

In [11]:
sys.path.append("../")
from Fast_Nqso import Fast_Nqso

In [12]:
mbright = {"u":14.7,
           "g":15.7,
           "r":15.8,
           "i":15.8,
           "z":15.3,
           "y":13.9
          }
def get_Nqso(bd, run, filter, wfd_mask=None):
    zmin = 0.3
    zmax = np.min([6.7, (lam_eff['LSST'+filter]/(912.*u.AA)-1.0).to(1.).value])
    for run_key in bd[run].keys():
        if run_key[1][-1]==filter:
            Key = run_key
            break
    mask = bd[run][Key].metricValues.mask
    if wfd_mask is not None:
        mask = mask | wfd_mask[run]
    mlim5 = bd[run][Key].metricValues[~mask]
    pix_area = bd[run][Key].slicer.pixArea*u.sr
    qso_counter = Fast_Nqso("LSST"+filter, "Shen20", "A", area=pix_area)
    Nqso = np.sum(qso_counter.Nqso(zmin, zmax, mbright[filter], mlim5))
    Area = len(mlim5) * pix_area.to(u.deg**2).value
    return Nqso, Area

In [13]:
#filters = ["u","g","r","i","z","y"]
filters = ["i"]
Nqso = dict()
Area = dict()
dens = dict()
for filter in filters:
    if filter not in Nqso:
        Nqso[filter] = np.zeros(len(dbRuns))
        Area[filter] = np.zeros(len(dbRuns))
        dens[filter] = np.zeros(len(dbRuns))
    for k, run in enumerate(dbRuns):
        Nqso[filter][k], Area[filter][k] = get_Nqso(bundleDicts, run, filter)
        dens[filter][k] = Nqso[filter][k]/Area[filter][k]

In [14]:
i_rank_sort = np.argsort(Nqso["i"])
print(Nqso["i"][i_rank_sort[0]], Nqso["i"][i_rank_sort[-1]])

12021824.932312284 17634855.182421736


In [15]:
fname = "Table_EM5_Nqso_i.txt"
tab_file = open(fname,"w")
for k,run in enumerate(list(bundleDicts.keys())):
    tab_file.write("{0:50s}".format(run))
    
    i_rank_percent = (np.argwhere(k==i_rank_sort)[0,0]+1)/len(Nqso["i"]) * 100.
    tab_file.write(" {0:10.2f} {1:5.1f}".format(Nqso["i"][k], i_rank_percent))
    tab_file.write(" {0:10.0f}".format(Area["i"][k]))
                   
    tab_file.write("\n")
tab_file.close()

### WFD filter

Now, we are going to filter to only consider areas with more than 750 visits, which are what constitutes the main portion of the WFD. 

In [16]:
WFDfp_folder_mafoutput = "WFDfootprint_{0:d}".format(NSIDE)
WFDfp_resultDbPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}'.format(
    your_username, WFDfp_folder_mafoutput)
WFDfp_metricDataPath = '/home/idies/workspace/Temporary/{0}/scratch/MAFOutput/{1}/MetricData/'.format(
    your_username, WFDfp_folder_mafoutput)

In [17]:
# get a dictionary of resultDb from given directory
WFDfp_resultDbs = getResultsDbs(WFDfp_resultDbPath)

In [18]:
# retrieve metricBundles for each opsim run and store them in a dictionary
WFDfp_bundleDicts = dict()
for runName in WFDfp_resultDbs:
    WFDfp_bundleDicts[runName] = bundleDictFromDisk(WFDfp_resultDbs[runName], runName, WFDfp_metricDataPath)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSID

In [19]:
#Load the WFD footprint for each OpSim run as a mask. 
Key = (1, 'nvisitsLong')
wfd_mask = dict()
for run in WFDfp_resultDbs:
    wfd_footprint = WFDfp_bundleDicts[run][Key].metricValues.filled(0)
    wfd_mask[run] = np.where(wfd_footprint > 750, False, True)

In [20]:
#filters = ["u","g","r","i","z","y"]
filters = ["i"]
Nqso_WFD = dict()
Area_WFD = dict()
dens_WFD = dict()
for filter in filters:
    if filter not in Nqso_WFD:
        Nqso_WFD[filter] = np.zeros(len(dbRuns))
        Area_WFD[filter] = np.zeros(len(dbRuns))
        dens_WFD[filter] = np.zeros(len(dbRuns))
    for k, run in enumerate(dbRuns):
        Nqso_WFD[filter][k], Area_WFD[filter][k] = get_Nqso(bundleDicts, run, filter, wfd_mask)
        dens_WFD[filter][k] = Nqso_WFD[filter][k]/Area_WFD[filter][k]

In [21]:
i_rank_sort = np.argsort(Nqso_WFD["i"])
print(Nqso_WFD["i"][i_rank_sort[0]], Nqso_WFD["i"][i_rank_sort[-1]])

6081435.785444825 13635493.063544396


In [22]:
dens_rank_sort = np.argsort(dens_WFD["i"])
print(dens_WFD["i"][dens_rank_sort[0]], dens_WFD["i"][dens_rank_sort[-1]])

610.6363384202693 627.5380734930592


In [23]:
fname = "Table_EM5_Nqso_i_WFD.txt"
tab_file = open(fname,"w")
for k,run in enumerate(list(bundleDicts.keys())):
    tab_file.write("{0:50s}".format(run))
    
    i_rank_percent = (np.argwhere(k==i_rank_sort)[0,0]+1)/len(Nqso_WFD["i"]) * 100.
    tab_file.write(" {0:10.2f} {1:5.1f}".format(Nqso_WFD["i"][k], i_rank_percent))
    tab_file.write(" {0:10.0f}".format(Area_WFD["i"][k]))
    tab_file.write(" {0:10.3f}".format(dens_WFD["i"][k]))
                   
    tab_file.write("\n")
tab_file.close()